In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif


In [11]:
!pip install gdown

In [12]:
import gdown

# ID del archivo de Google Drive
file_id = '1q7F9-9Pkz8nn14dBxSYUGsCWA4proIBN'
gdown.download(f'https://drive.google.com/uc?id={file_id}', '/content/flights_preprocess.csv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1q7F9-9Pkz8nn14dBxSYUGsCWA4proIBN
From (redirected): https://drive.google.com/uc?id=1q7F9-9Pkz8nn14dBxSYUGsCWA4proIBN&confirm=t&uuid=699f1cf7-6372-4af9-bdc0-0dd94e30011d
To: /content/flights_preprocess.csv
100%|██████████| 1.37G/1.37G [00:22<00:00, 59.7MB/s]


'/content/flights_preprocess.csv'

In [13]:
# Llama a la base preprocesada
data = pd.read_csv('/content/flights_preprocess.csv')



In [14]:
# debido al tamaño de los datos, con el objetivo de optimizar recursos, vamos a trabajar con una muestra

data = data.sample(frac=0.1, random_state=42)

In [15]:
# definimos nuestras variables explicativas y objetivo (Arr_Delayed - si los vuelos se retrasaron o no)

X = data.drop(columns=['Arr_Delayed', 'FlightDate'])
y = data['Arr_Delayed']

In [16]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# sabemos que existe un desbalanceo en las etiquetas (muy pocos vuelos llegan retrasados). Por ello, vamos a
# balancear generando ejemplos sinteticos de vuelos atrasados por medio de SMOTE

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [18]:
# verificamos

print(pd.Series(y_train).value_counts())

Arr_Delayed
False    362250
True     362250
Name: count, dtype: int64


In [19]:
# vamos a reducir la dimensionalidad de nuestras caracteristicas por medio de componentes principales (PCA)

pca = PCA(n_components= 3)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [20]:
# ahora vamos a entrenar distintos modelos. Nuestro baseline model va a ser una regresión logística, mientras que los demás modelos que entrenaremos serán:
# Random Forest, Gradient Boosting, Support Vector Machine

model_log = LogisticRegression(max_iter=1000).fit(X_train_pca, y_train)
model_rf = RandomForestClassifier(random_state=42).fit(X_train_pca, y_train)


In [21]:
model_gb = GradientBoostingClassifier(random_state=42).fit(X_train_pca, y_train)

In [22]:
# model_svm = SVC(random_state=42).fit(X_train_pca, y_train)

In [23]:
# definimos una funcion para generar las métricas

def metricas(model):
    y_pred = model.predict(X_test_pca)
    metrics = {
        "Model": "{}".format(model),
        "Accuracy": accuracy_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred)
    }
    return metrics

In [24]:
# usemos la funcion con los distintos modelos y generemos una lista

resultados = []
metrica_log = metricas(model_log)
metrica_rf = metricas(model_rf)
metrica_gb = metricas(model_gb)
#metrica_svm = metricas(model_svm)
resultados.append(metrica_log)
resultados.append(metrica_rf)
resultados.append(metrica_gb)
# resultados.append(metrica_svm)

In [25]:
# vamos a comparar los modelos según las distintas metricas:

resultados_df = pd.DataFrame(resultados)

# por accuracy

print("Comparación de Modelos: Accuracy")
print(resultados_df.sort_values(by="Accuracy", ascending=False))

Comparación de Modelos: Accuracy
                                         Model  Accuracy  F1 Score  Precision  \
1      RandomForestClassifier(random_state=42)  0.756662  0.396130   0.373940   
0            LogisticRegression(max_iter=1000)  0.521569  0.273979   0.192294   
2  GradientBoostingClassifier(random_state=42)  0.471270  0.301341   0.201011   

     Recall  
1  0.421120  
0  0.476308  
2  0.601627  


In [26]:
# por F1 Score

print("Comparación de Modelos: F1 Score")
print(resultados_df.sort_values(by="F1 Score", ascending=False))

Comparación de Modelos: F1 Score
                                         Model  Accuracy  F1 Score  Precision  \
1      RandomForestClassifier(random_state=42)  0.756662  0.396130   0.373940   
2  GradientBoostingClassifier(random_state=42)  0.471270  0.301341   0.201011   
0            LogisticRegression(max_iter=1000)  0.521569  0.273979   0.192294   

     Recall  
1  0.421120  
2  0.601627  
0  0.476308  


In [27]:
# por Precision

print("Comparación de Modelos: Precision")
print(resultados_df.sort_values(by="Precision", ascending=False))

Comparación de Modelos: Precision
                                         Model  Accuracy  F1 Score  Precision  \
1      RandomForestClassifier(random_state=42)  0.756662  0.396130   0.373940   
2  GradientBoostingClassifier(random_state=42)  0.471270  0.301341   0.201011   
0            LogisticRegression(max_iter=1000)  0.521569  0.273979   0.192294   

     Recall  
1  0.421120  
2  0.601627  
0  0.476308  


In [28]:
# por Recall

print("Comparación de Modelos: Recall")
print(resultados_df.sort_values(by="Recall", ascending=False))

Comparación de Modelos: Recall
                                         Model  Accuracy  F1 Score  Precision  \
2  GradientBoostingClassifier(random_state=42)  0.471270  0.301341   0.201011   
0            LogisticRegression(max_iter=1000)  0.521569  0.273979   0.192294   
1      RandomForestClassifier(random_state=42)  0.756662  0.396130   0.373940   

     Recall  
2  0.601627  
0  0.476308  
1  0.421120  


El modelo debe quedar como

/flight_delay_prediction
    ├── app.py                   # Archivo principal para la API
    ├── model.pkl                # Modelo entrenado guardado
    ├── requirements.txt         # Lista de dependencias
    └── data/                    # Carpeta para datos (opcional)

In [29]:
!pip install flask scikit-learn joblib gdown

In [30]:
import joblib

# Guardar el mejor modelo (Random Forest)
joblib.dump(model_rf, 'model.pkl')

['model.pkl']

In [ ]:
from flask import Flask, request, jsonify
import os

app = Flask(__name__)

# Cargar el modelo entrenado
model_path = os.path.join(os.getcwd(), 'model.pkl')  # Asegúrate de que este sea el path correcto
model = joblib.load(model_path)

@app.route('/predict', methods=['POST'])
def predict():
    # Obtener los datos del cuerpo de la solicitud
    data = request.get_json(force=True)

    # Asegúrate de que los datos recibidos sean correctos (ajusta según tus características)
    features = np.array(data['features']).reshape(1, -1)  # Ajusta según las características esperadas

    # Realizar la predicción
    prediction = model.predict(features)

    return jsonify({'prediction': bool(prediction[0])})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
python app.py

In [ ]:
curl -X POST http://127.0.0.1:5000/predict -H "Content-Type: application/json" -d '{"features": [value1, value2, value3, ...]}'